# 07 - Création d'un dataset de la fusion des datasets de clean metrics et clean jobs/job_events </br> identification d'évènements 'Defaut', 'Bourrage' et 'Erreur' en criticité ERROR

Ce notebook génère 1 csv :

- dataset_for_preprocess.csv qui fusionne les données du dataset de metrics et jobs/job_events

Etapes : 

B) Import des datasets : 

- metrics est quasi-brut (la colonne events est fractionnée et tous les identifiants d'évènements sont des nombres)

- jobs (le dataset fusionné des données de jobs et job_events) dont chaque ligne est un job, un job est unique et il a un début et une fin

C) Réduction du dataset jobs et réductions succesives du dataset metrics (les lignes évènements avec certains identifiants)

D) Concaténation des datasets jobs et metrics (suppression des lignes inutiles : toutes celles de jobs et metrics si aucun jobId associé)


# A. Imports

## a) Librairies

In [1]:
import os, json, ast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## b) Datasets

In [2]:
# source path to datasets
path = '../data/'
metrics = 'metrics/clean_merge_metrics_dataset.csv'
jobs = 'jobs/merge_raw_jobs_and_clean_jobevents_dataset.csv'

save_csv = '../data/dataset_for_preprocess_id_events_filtered_07.csv'

# B. Jeux de données

### a) Metrics

In [3]:
# création d'un dataframe à partir du csv de données
metrics_df = pd.read_csv(os.path.join(path, metrics), index_col=0)
print(f'metrics dataset shape {metrics_df.shape}')

metrics dataset shape (761371, 14)


In [4]:
# la colonne timestamp contient-elle des valeurs en double ?
metrics_df['timestamp'].duplicated().any()

True

In [5]:
metrics_df['timestamp'].min()

'2022-11-03 08:43:13.960000+00:00'

In [6]:
# Grouper les lignes par la colonne "timestamp" et obtenir les index correspondants
groupes = metrics_df.groupby('timestamp').groups
print(len(groupes.keys()))

276152


In [7]:
metrics_df.head(3)

,source_events,timestamp,criticality_events,identification_events,name_modules,type_modules,generation_modules,name_counters_modules,value_counters_modules,name_connected_operators,level_connected_operators,status,varnishLevelsTargetvolume,varnishLevelsTotalvolume
0,iFoil,2022-11-03 08:43:16.808000+00:00,INFO,407,iFoil L,iFoil,Gen. 2,Total Pages Counter,31185,User,Operator,WARNING,1386,18000
1,iFoil,2022-11-03 08:43:16.808000+00:00,INFO,407,iFoil L,iFoil,Gen. 2,Foiled Pages Counter,79566,User,Operator,WARNING,1386,18000
2,PLC,2022-11-03 08:43:16.746000+00:00,WARNING,407,Print Engine 1,Varnish Printer,NaN,3D Varnish Counter,308536,User,Operator,WARNING,1386,18000


### b) Jobs

In [8]:
# création d'un dataframe à partir du csv de données
jobs_df = pd.read_csv(os.path.join(path, jobs), index_col=0)
print(f'jobs dataset shape {jobs_df.shape}')

jobs dataset shape (4182, 45)


In [9]:
# la colonne started_at contient-elle des valeurs en double ?
jobs_df['started_at'].duplicated().any()

False

In [10]:
jobs_df['started_at'].min()

'2022-10-25 08:25:14.728000+00:00'

In [11]:
jobs_df.head(3)

,started_at,ended_at,paperHeight_job,paperWidth_job,scanner_mode,bars_job,run,total_run,copies_per_run,jobId,...,redScore_gridMode_remoteScannerRegistration,redScore_cropmarksMode_remoteScannerRegistration,redScore_fullScannerMode_remoteScannerRegistration,blueScore_fullScannerMode_remoteScannerRegistration,greenScore_fullScannerMode_remoteScannerRegistration,enable_specialSubstrate_remoteScannerRegistration,mode_remoteScannerRegistration,jobState,total_copies,varnishConsumptionVarnish_3d
0,2022-10-25 08:25:14.728000+00:00,2022-10-25 08:26:18.522000+00:00,500,320,1,2,0,0,0,1666686314,...,1500,1500,1500,16,16,False,1,SUCCESS,1,0.046652
1,2022-10-25 08:27:37.387000+00:00,2022-10-25 08:28:49.018000+00:00,500,320,1,2,0,0,0,1666686457,...,1500,1500,1500,16,16,False,1,SUCCESS,4,0.158389
2,2022-10-25 08:30:20.210000+00:00,2022-10-25 08:31:20.859000+00:00,500,320,1,2,0,0,0,1666686620,...,1500,1500,1500,16,16,False,1,SUCCESS,1,0.368169


Les tailles des datasets sont déséquilibrés :

- 3510431 lignes pour metrics

- 16295 lignes pour jobs

Les dates de début sont différentes :

- '2022-04-15 05:55:06.678000+00:00' pour metrics

- '2021-06-18 09:22:46.866000+00:00' pour jobs

Le dataset metrics compte 1242037 doublons pour la colonne timestamp

# C. Equilibrage des jeux de données

## a) Réduction de jobs

In [12]:
# concordance des données de temps dans un même cadre
jobs_reduced = jobs_df[jobs_df.started_at > metrics_df.timestamp.min()]
jobs_reduced.reset_index()
jobs_reduced.shape

(4101, 45)

## b) Réduction de metrics

### Par criticité ERROR

In [13]:
metrics_reduced = metrics_df[metrics_df['criticality_events'] == 'ERROR']

### Par importance d'identifiant d'évènement

In [14]:
# lignes avec des évènements identifiant une maintenance
id_to_drop_1 = [391, 330, 377, 407, 332, 331, 313, 333, 376, 372, 344, 343, 371, 358, 334, 311, 472, 408, 406, 350, 2, 352, 346, 0]
# lignes avec des évènements identifiant une intervention humaine
id_to_drop_2 = [352, 324, 381, 440, 385, 405, 447, 388, 320, 417, 444, 329, 315, 384, 345, 349, 466, 419]
# lignes avec des évènements identifiant 'arrêt d'urgence'
id_to_drop_3 = [321, 322, 323, 379, 380]
# fusion des listes d'identifiants à supprimer
id_to_drop = id_to_drop_1 + id_to_drop_2 + id_to_drop_3
# on supprime les lignes avec des évènements identifiant une maintenance
metrics_reduced = metrics_reduced[~metrics_reduced['identification_events'].isin(id_to_drop)]
metrics_reduced.shape

(1778, 14)

In [15]:
# on vérifie la distribution par identifiant d'évènements
metrics_reduced['identification_events'].value_counts()

454     415
1001    293
480     237
357     134
445     122
355     116
356      82
383      78
446      47
354      45
325      40
386      38
453      33
387      27
451      25
328      21
430       6
1000      4
460       4
382       3
418       3
1002      2
389       2
416       1
Name: identification_events, dtype: int64

### Par nombre d'identifiant d'évènement

In [16]:
# # Calculer le nombre de valeurs d'identification_events pour chaque groupe
# counts = metrics_by_identification.groupby('identification_events')['identification_events'].transform('count')
# # Filtrer les lignes où le nombre de valeurs d'identification_events est inférieur à 20
# metrics_reduced = metrics_by_identification[counts >= 50]

In [17]:
# # on vérifie la distribution par identification_events
# metrics_reduced['identification_events'].value_counts()

### Premier équilibrage par identifiant d'évènement

In [18]:
# fonction permettant de reduire le nombre de ligne d'une classe à n lignes
def balance_dataframe_by_identification(df, identification, n):
    # on filtre les lignes du df qui ont la valeur criticality dans la colonne 'criticality_events'
    identification_df = df[df['identification_events'] == identification]
    # on selectionne un nombre aléatoire de lignes
    identification_df_sample = identification_df.sample(n=n)
    # on stoke les index des lignes du df qui ne sont pas dans le sample
    index_to_delete = identification_df[~identification_df.index.isin(identification_df_sample.index)].index
    # suppression des lignes exclu du sample
    return df.drop(index_to_delete)

In [19]:
# # on réduit le nombre de classe pour l'id 0
# metrics_reduced = balance_dataframe_by_identification(metrics_reduced,0,2845)
# # on vérifie la distribution par identification_events
# metrics_reduced['identification_events'].value_counts()

In [20]:
# # on vérifie la distribution par criticité
# metrics_reduced['criticality_events'].value_counts()

# D. Création du datatset pour le pre-processing

In [21]:
metrics_reduced['jobId'] = 0
metrics_reduced.shape

(1778, 15)

## a) Analyse des dates

In [22]:
# # Vérifier si des dates communes existent
# dates_communes_exist = jobs_reduced['started_at'].isin(metrics_reduced['timestamp']).any()

# # Afficher le résultat
# if dates_communes_exist:
#     # Filtrer les dates communes
#     dates_communes = jobs_df['started_at'][jobs_df['started_at'].isin(metrics_df['timestamp'])]
    
#     # Compter le nombre de dates communes
#     nombre_dates_communes = len(dates_communes)
    
#     print(f"Des dates communes existent entre les ensembles de données: {nombre_dates_communes}")
# else:
#     print("Aucune date commune n'a été trouvée entre les ensembles de données.")


In [23]:
# création de dictionnaire du nombre de lignes par mois
def compare_datetime_series_shapes(s1, s2):
    s1_serie = {}
    s2_serie = {}
    s1 = pd.to_datetime(s1)
    s2 = pd.to_datetime(s2)
    for i in range(1,13):
        s1_month = s1[s1.dt.month == i]
        s2_month = s2[s2.dt.month == i]
        print(i, s1_month.shape, s2_month.shape)
        s1_serie[i] = s1_month.index
        s2_serie[i] = s2_month.index
    return s1_serie, s2_serie

In [24]:
# on liste les index des lignes par mois
#jobs_indexes_by_month, metrics_indexes_by_month = compare_datetime_series_shapes(jobs_reduced['started_at'], metrics_reduced['timestamp'])

In [25]:
# nombre de lignes par mois
# month = 5

# Selection des lignes de jobs en Mai
# jobs_df['started_at'] = pd.to_datetime(jobs_df['started_at'])
# jobs_batch = jobs_df[ (jobs_df['started_at'].dt.month == month)]
# jobs_batch = jobs_df.loc[jobs_df.index.isin(jobs_indexes_by_month.get(month))]
# jobs_batch.shape

# Selection des lignes de metrics en Mai
# metrics_df['timestamp'] = pd.to_datetime(metrics_df['timestamp'])
# metrics_batch = metrics_df[ (metrics_df['timestamp'].dt.month == month)]
# metrics_batch = metrics_df.loc[metrics_df.index.isin(metrics_indexes_by_month.get(month))]
# metrics_batch.shape

## b) Correspondance de dates entre metrics et jobs

### Pour chaque ligne de metrics quel est le jobId correspondant

In [26]:
import warnings
warnings.simplefilter("ignore")

In [27]:
# DataFrame 1 
# avec les intervalles de temps
dataframe1 = jobs_reduced[['started_at', 'ended_at', 'jobId']]
# conversion des colonnes au format datetime
cols_to_convert = ['started_at', 'ended_at']
dataframe1[cols_to_convert] = dataframe1[cols_to_convert].apply(pd.to_datetime)
dataframe1 = dataframe1.sort_values('started_at')
dataframe1 = dataframe1.reset_index()
dataframe1.head(3)

,index,started_at,ended_at,jobId
0,81,2022-11-03 08:46:43.943000+00:00,2022-11-03 08:48:43.830000+00:00,1667465203
1,82,2022-11-03 08:49:11.135000+00:00,2022-11-03 08:49:54.745000+00:00,1667465350
2,83,2022-11-03 08:50:46.125000+00:00,2022-11-03 08:52:05.703000+00:00,1667465445


In [28]:
# DataFrame 2 
# avec les valeurs de date
dataframe2 = pd.DataFrame({
    'timestamp': metrics_reduced['timestamp'].values,
    'criticality_events': metrics_reduced['criticality_events'].values,
    'identification_events': metrics_reduced['identification_events'].values,
    'jobId': 0
}, index=None)
# conversion des colonnes au format datetime
dataframe2['timestamp'] = pd.to_datetime(dataframe2['timestamp'])
dataframe2 = dataframe2.sort_values('timestamp')
dataframe2 = dataframe2.reset_index()
dataframe2.head(3)

,index,timestamp,criticality_events,identification_events,jobId
0,550,2022-11-03 08:48:47.596000+00:00,ERROR,1001,0
1,551,2022-11-03 08:49:58.465000+00:00,ERROR,1001,0
2,552,2022-11-03 08:59:29.480000+00:00,ERROR,454,0


In [29]:
import time
# début du temps d'execution de la cellule
start = time.time()

# Parcours chaque ligne de job
for index1, row1 in dataframe1.iterrows():
    # creation des variables
    job_starts = row1.started_at.tz_localize(None)
    job_ends = row1.ended_at.tz_localize(None)

    # mois de référence
    month = job_starts.month
    if month != job_ends.month :
        dataframe2_by_month = dataframe2[(dataframe2['timestamp'].dt.month == month) | (dataframe2['timestamp'].dt.month == job_ends.month) & (dataframe2['jobId'] == 0)]
        dataframe2_by_month = dataframe2_by_month.sort_values('timestamp')
    else :
        dataframe2_by_month = dataframe2[(dataframe2['timestamp'].dt.month == month) & (dataframe2['jobId'] == 0)]
        dataframe2_by_month = dataframe2_by_month.sort_values('timestamp')

    for index2, row2 in dataframe2_by_month.iterrows():
        # creation des variables
        metrics_timestamp = row2['timestamp'].tz_localize(None)
        # si le timestamp de la lignes de row2 de metrics est compris dans l'intervalle de temps du job de row1
        if metrics_timestamp >= job_starts :
            if metrics_timestamp <= job_ends :
                dataframe2.loc[index2, 'jobId'] = dataframe1.loc[index1, 'jobId']
    # # avant de passer à la row1 suivante
    # # on affiche l'index1 toutes les 1000 lignes et son temps d'execution
    print(f'[ job index {index1} = time {time.time() - start}]') if index1 % 1000 == 0 and index1 != 0 else None


[ job index 1000 = time 12.829029083251953]
[ job index 2000 = time 21.98390030860901]
[ job index 3000 = time 27.385176181793213]
[ job index 4000 = time 33.84590792655945]


In [30]:
# on vérifie la distribution par identification_events pour les lignes du dataframe2 (metrics_df) qui ont été associées à un jobId
dataframe2[dataframe2['jobId'] != 0]['identification_events'].value_counts()

454    277
355     56
357     49
356     34
451     21
354     17
453      8
325      6
387      4
383      2
446      2
445      1
386      1
Name: identification_events, dtype: int64

In [31]:
print(f'Nombre de ligne de metrics : {dataframe2.shape[0]} (dont {dataframe2[dataframe2.jobId != 0].shape[0]} avec un jobId associé)')

Nombre de ligne de metrics : 1778 (dont 478 avec un jobId associé)


## c) Fusion des dataframes jobs et metrics

In [32]:
merged_df = jobs_reduced.merge(dataframe2, on='jobId')
merged_df.shape

(478, 49)

In [33]:
# on vérifie la distribution par criticité
merged_df['identification_events'].value_counts()

454    277
355     56
357     49
356     34
451     21
354     17
453      8
325      6
387      4
383      2
446      2
445      1
386      1
Name: identification_events, dtype: int64

In [34]:
# on vérifie la distribution par criticité
merged_df['criticality_events'].value_counts()

ERROR    478
Name: criticality_events, dtype: int64

### Dataframe ne contenant que des criticality = ERROR

In [35]:
# merged_df_error = merged_df[merged_df.criticality_events == 'ERROR']

In [36]:
# on vérifie la distribution par criticité
# merged_df_error['identification_events'].value_counts()

### Equilibrage des classes identification_events en fonction de l'id le plus fréquent des lignes ERROR

In [37]:
# # on identifie la classe identification_events la plus fréquente dans les ERROR
# identification_counts = merged_df[merged_df['criticality_events'] == 'ERROR']['identification_events'].value_counts()
# most_frequent_identification_event = identification_counts.idxmax()
# nombre_lignes = identification_counts[most_frequent_identification_event]
# print(f"L'id '{most_frequent_identification_event}' est le plus fréquent des lignes ERROR avec {nombre_lignes} lignes.")

## d) Output

In [38]:
merged_df.to_csv(path_or_buf=save_csv)